# 2. Lists and Dictionaries

## Item 11: Know How to Slice Sequences

In [2]:
a = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h']

In [3]:
a[:]

['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h']

In [4]:
a[:-1]

['a', 'b', 'c', 'd', 'e', 'f', 'g']

In [5]:
a[2:-1]

['c', 'd', 'e', 'f', 'g']

In [6]:
a[-3:-1]

['f', 'g']

In [7]:
a[-20:]

['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h']

In [8]:
b = a[3:]
b[1] = 99
b

['d', 99, 'f', 'g', 'h']

In [9]:
a # a is not changed.

['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h']

In [10]:
b = a[:]
assert b == a and b is not a

In [11]:
b = a
a[:] = ['new']

assert b is a

a = ['new']

assert b is not a

In [12]:
a = [1, 2, 3]
a[1:] = [4]
a

[1, 4]

## Item 12: Avoid Striding and Slicing in a Single Expression

Avoid these kind of expressions as they are confusing.
```python
x = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h']
x[2::2]     # ['c', 'e', 'g']
x[-2::-2]   # ['g', 'e', 'c', 'a']
x[-2:2:-2]  # ['g', 'e']
x[2:2:-2]   # []
```

## Item 13: Prefer Catch-All Unpacking Over Slicing

In [13]:
car_ages = [0, 9, 4, 8, 7, 20, 19, 1, 6, 15]
car_ages_descending = sorted(car_ages, reverse=True)

In [14]:
oldest, second_oldest, *others = car_ages_descending
print(oldest, second_oldest, others)

20 19 [15, 9, 8, 7, 6, 4, 1, 0]


In [16]:
oldest, *others, youngest = car_ages_descending
print(oldest, youngest, others)

20 0 [19, 15, 9, 8, 7, 6, 4, 1]


In [17]:
car_inventory = {
    'Downtown': ('Silver Shadow', 'Pinto', 'DMC'),
    'Airport': ('Skyline', 'Viper', 'Gremlin', 'Nova'),

}

((loc1, (best1, *rest1)),
 (loc2, (best2, *rest2))) = car_inventory.items()
print(f'Best at {loc1} is {best1}, {len(rest1)} others')
print(f'Best at {loc2} is {best2}, {len(rest2)} others')

Best at Downtown is Silver Shadow, 2 others
Best at Airport is Skyline, 3 others


In [18]:
it = iter(range(1, 10))
one, two, *others = it
print(one, two, others)

1 2 [3, 4, 5, 6, 7, 8, 9]


## Item 14: Sort by Complex Criteria Using the key Parameter

In [19]:
numbers = [93, 86, 11, 68, 70]
numbers.sort()
print(numbers)

[11, 68, 70, 86, 93]


In [20]:
class Tool:
    def __init__(self, name, weight):
        self.name = name
        self.weight = weight

    def __repr__(self):
        return f'Tool({self.name!r}, {self.weight})'

tools = [
    Tool('level', 3.5),
    Tool('hammer', 1.25),
    Tool('screwdriver', 0.5),
    Tool('chisel', 0.25),
]

In [26]:
print('Unsorted:', repr(tools))
tools.sort(key=lambda x: x.name)
print('Sorted: ', tools)

Unsorted: [Tool('chisel', 0.25), Tool('hammer', 1.25), Tool('level', 3.5), Tool('screwdriver', 0.5)]
Sorted:  [Tool('chisel', 0.25), Tool('hammer', 1.25), Tool('level', 3.5), Tool('screwdriver', 0.5)]


In [27]:
saw = (5, 'circular saw')
jackhammer = (40, 'jackhammer')
zzz = (40, 'zzz')

# If the first position in the tuples being compared are 
#  equal—weight in this case—then the tuple comparison 
#  will move on to the second position, and so on:
assert jackhammer > saw 
assert zzz > jackhammer

In [29]:
power_tools = [
    Tool('drill', 4),
    Tool('circular saw', 5),
    Tool('jackhammer', 40),
    Tool('sander', 4),
]

power_tools.sort(key=lambda x: (x.weight, x.name))
print(power_tools)

[Tool('drill', 4), Tool('sander', 4), Tool('circular saw', 5), Tool('jackhammer', 40)]


In [30]:
power_tools.sort(key=lambda x: (-x.weight, x.name))
print(power_tools)

[Tool('jackhammer', 40), Tool('circular saw', 5), Tool('drill', 4), Tool('sander', 4)]


In [32]:
# Python provides a stable sorting algorithm (Timsort).
power_tools.sort(key=lambda x: x.name) # Name ascending
power_tools.sort(key=lambda x: x.weight, # Weight descending
                 reverse=True)
print(power_tools)

[Tool('jackhammer', 40), Tool('circular saw', 5), Tool('drill', 4), Tool('sander', 4)]


## Item 15: Be Cautious When Relying on dict Insertion Ordering

In [35]:
votes = {
    'otter': 1281,
    'polar bear': 587,
    'fox': 863,
}


def populate_ranks(votes, ranks):
    names = list(votes.keys())
    names.sort(key=votes.get, reverse=True)
    for i, name in enumerate(names, 1):
        ranks[name] = i
        

def get_winner(ranks):
    return next(iter(ranks))

In [39]:
ranks = {}
populate_ranks(votes, ranks)
print(ranks)
winner = get_winner(ranks)
print('Winner:', winner)

{'otter': 1, 'fox': 2, 'polar bear': 3}
Winner: otter


In [40]:
from collections.abc import MutableMapping

class SortedDict(MutableMapping):
    '''
    To sort by the name.
    '''
    def __init__(self):
        self.data = {}

    def __getitem__(self, key):
        return self.data[key]
    def __setitem__(self, key, value):
        self.data[key] = value

    def __delitem__(self, key):
        del self.data[key]
    def __iter__(self):
        keys = list(self.data.keys())
        keys.sort()
        for key in keys:
            yield key
    def __len__(self):
          return len(self.data)

In [54]:
sorted_ranks = SortedDict()
populate_ranks(votes, sorted_ranks)
print(sorted_ranks.data)
winner = get_winner(sorted_ranks)
print('''
# This is unexpected. 
# This happens as get_winner() function assumes
#  the argument is dict not SortedDict.
''', 'Winner: ', winner) 

{'otter': 1, 'fox': 2, 'polar bear': 3}

# This is unexpected. 
# This happens as get_winner() function assumes
#  the argument is dict not SortedDict.
 Winner:  fox


In [44]:
def get_winner(ranks):
    for name, rank in ranks.items():
        if rank == 1:
            return name

winner = get_winner(sorted_ranks)
print(winner)

otter


Typing could've prevented the unexpected behavior.

```python
from typing import Dict, MutableMapping

def populate_ranks(votes: Dict[str, int],
                   ranks: Dict[str, int]) -> None:
    names = list(votes.keys())
    names.sort(key=votes.get, reverse=True)
    for i, name in enumerate(names, 1):
        ranks[name] = i

def get_winner(ranks: Dict[str, int]) -> str:
    return next(iter(ranks))

votes = {
    'otter': 1281,
    'polar bear': 587,
    'fox': 863,
}
sorted_ranks = SortedDict()
populate_ranks(votes, sorted_ranks)
print(sorted_ranks.data)
winner = get_winner(sorted_ranks)
print(winner)
```

```sh
$ python3 -m mypy --strict example.py
.../example.py:48: error: Argument 2 to "populate_ranks" has
➥incompatible type "SortedDict"; expected "Dict[str, int]"
.../example.py:50: error: Argument 1 to "get_winner" has
➥incompatible type "SortedDict"; expected "Dict[str, int]"
```

## Item 16: Prefer get Over in and KeyError to Handle Missing Dictionary Keys

Do this:

```python
count = counters.get(key, 0)
counters[key] = count + 1
```

Not this:
```python
if key not in counters:
    counters[key] = 0
counters[key] += 1

if key in counters:
    counters[key] += 1
else:
    counters[key] = 1

try:
    counters[key] += 1
except KeyError:
    counters[key] = 1
```

When you want to add a list to a dict, you can do this but it's not very readable.

```python
names = votes.setdefault(key, [])
names.append(who)
```

You could do these instead:

```python
if key in votes:
    names = votes[key]
else:
    votes[key] = names = []
    
names.append(who)

...
    
    
try:
    names = votes[key]
except KeyError:
    votes[key] = names = []

names.append(who)

...

names = votes.get(key)
if names is None:
    votes[key] = names = []

names.append(who)

...

if (names := votes.get(key)) is None:
    votes[key] = names = []

names.append(who)
```

### Item 17: Prefer defaultdict Over setdefault to Handle Missing Items in Internal State

Do this:
```python
from collections import defaultdict

class Visits:
    def __init__(self):
       self.data = defaultdict(set)

    def add(self, country, city):
       self.data[country].add(city)

visits = Visits()
visits.add('England', 'Bath')
visits.add('England', 'London')
print(visits.data)

>>>
defaultdict(<class 'set'>, {'England': {'London', 'Bath'}})
```

Instead of:
```python
class Visits:
    def __init__(self):
        self.data = {}

    def add(self, country, city):
        city_set = self.data.setdefault(country, set())
        city_set.add(city)
```

Since the `setdefault` method is confusing and this constructs a new set instance on every call.

## Item 18: Know How to Construct Key-Dependent Default Values with `__missing__`

To improve this:
```python
pictures = {}
path = 'profile_1234.png'

if (handle := pictures.get(path)) is None:
    try:
        handle = open(path, 'a+b')
    except OSError:
        print(f'Failed to open path {path}')
        raise
    else:
        pictures[path] = handle

handle.seek(0)
image_data = handle.read()
```

You can use `__missing__`:
```
```